#### Day 5 - B
Input is made of rules and updates. The rules dictates which updates are valid by imposing an order on which update items can precede another item.

Fix the invalid updates and sum the middle entry of each invalid update.

In [94]:
#Import Libraries and settings

settings = {
    "day": 5,
    "test_data": 0
}

In [95]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    #Split input between rules and updates
    rules = []
    updates = []
    reading_rules = True
    for line in lines:
        if line == "":
            reading_rules = False
        elif reading_rules:
            rules.append(line)
        else:
            line_arr = line.split(",")
            line_num = [int(x) for x in line_arr]
            updates.append(line_num)


    return rules, updates

rules_arr, updates = load_input(settings)

In [96]:
rules_arr[0:5]

['91|88', '92|39', '92|57', '32|64', '32|59']

In [97]:
updates[5]

[96, 43, 78, 17, 63, 81, 54, 64, 72, 53, 35]

In [98]:
#Convert the array of rules into a dictionary
def build_rules_dict(rules):
    rules_dict = {}
    
    for rule in rules:
        values = rule.split("|")
        key = int(values[0])
        value = int(values[1])

        if key not in rules_dict.keys():
            rules_dict[key] = []
        
        rules_dict[key].append(value)

    return rules_dict

rules = build_rules_dict(rules_arr)

In [99]:
#Function to process an update line
def process_update(rules, update):
    #Iterate through update
    for idx, item in enumerate(update):
        processed = update[:idx]

        #Check if the current item has rules
        if item in rules.keys():
            rules_for_item = rules[item]

            #Check if any previous items violate the rules for the current one
            if any(e in rules_for_item for e in processed):
                return False

    return True

#Determine if an item is safe to insert by considering the other items waiting to insert
def safe_to_insert(rules, item, pending):
    for future_item in pending:
        if future_item in rules.keys():
            if item in rules[future_item]:
                return False

    return True

#Sort an invalid update into the correct order
def fix_update(rules, update):

    #Temp vairables to iterate over update values
    to_insert = update
    fixed_update = []
    idx_arr = 0

    #Loop until all items are inserted
    while to_insert:
        #Next item to consider inserting
        next_item = to_insert[idx_arr]

        #Check if the item is safe to insert
        if safe_to_insert(rules, next_item, to_insert[:idx_arr] + to_insert[idx_arr+1:]):
            #Insert the item and remove it from the to_insert list
            fixed_update.append(next_item)
            to_insert = to_insert[:idx_arr] + to_insert[idx_arr+1:]

            #If the last item from the to_insert list was inserted, reset the array index
            if idx_arr == len(to_insert):
                idx_arr = 0

        else:

            #Update idx_arr for the next item
            if idx_arr < len(to_insert) - 1:
                idx_arr += 1
            else:
                idx_arr = 0

    return fixed_update


#Function to iterate over each update and maintain a running total
def process_all_updates(rules, updates):
    sum_valid = 0

    for update in updates:
        #If the update is not valid
        if not process_update(rules, update):
            fixed_update = fix_update(rules, update)
            middle_item = fixed_update[int((len(fixed_update)-1)/2)]
            sum_valid += middle_item
            
    return sum_valid

In [100]:
process_all_updates(rules, updates)

4828